## Exploring fluxes

In [ ]:
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('model_atocopherol.xml')
model.objective = model.reactions.DM_avite1_c

To get a better understanding of how to improve the production pathway, the associated reactions and fluxes are explored. The metabolite 3-(4-Hydroxyphenyl)pyruvate is the starting point for the heterologous pathway that was added to the model.

In [85]:
model.metabolites.get_by_id('34hpp_c')

Metabolite identifier,34hpp_c
Name,3-(4-Hydroxyphenyl)pyruvate
Memory address,0x07ff88828e4f0
Formula,C9H7O4
Compartment,c
In 3 reaction(s),"TYRTA, 34HPPOR, PPND"


3-(4-Hydroxyphenyl)pyruvate is part of three reactions. 34HPPOR is the first reaction of the heterologous pathway that consumes 3-(4-Hydroxyphenyl)pyruvate. Whereas TYRTA and PPND are reactions where 3-(4-Hydroxyphenyl)pyruvate is produced.

In [71]:
model.reactions.PPND

Reaction identifier,PPND
Name,Prephenate dehydrogenase
Memory address,0x07ff889913df0
Stoichiometry,nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c Nicotinamide adenine dinucleotide + Prephenate --> 3-(4-Hydroxyphenyl)pyruvate + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced
GPR,b2600
Lower bound,0.0
Upper bound,1000.0


In [86]:
model.reactions.TYRTA

Reaction identifier,TYRTA
Name,Tyrosine transaminase
Memory address,0x07ff88a2b7880
Stoichiometry,akg_c + tyr__L_c <=> 34hpp_c + glu__L_c 2-Oxoglutarate + L-Tyrosine <=> 3-(4-Hydroxyphenyl)pyruvate + L-Glutamate
GPR,b4054 or b0928 or b3770
Lower bound,-1000.0
Upper bound,1000.0


While exploring the fluxes, the production of biomass is set to half of the optimal production rate as a compromise between cell growth and production of alpha-tocopherol.

In [80]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    solution = model.optimize()
print("EX_tyr__L_e", solution.fluxes.EX_tyr__L_e)
print("PPND", solution.fluxes.PPND)
print("TYRTA",solution.fluxes.TYRTA)
print("HGPHT",solution.fluxes.HGPHT)
print("TOCOPHOM1",solution.fluxes.TOCOPHOM1)
print("DM_avite1_c",solution.fluxes.DM_avite1_c)

EX_tyr__L_e 0.0
PPND 0.6741627607421248
TYRTA -0.06077236
HGPHT 0.6133904007421248
TOCOPHOM1 0.6133904007421248
DM_avite1_c 0.6133904007421248


In [58]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    solution = model.optimize()
print("EX_tyr__L_e", solution.fluxes.EX_tyr__L_e)
print("PPND", solution.fluxes.PPND)
print("TYRTA",solution.fluxes.TYRTA)
print("TOCOPHOM1",solution.fluxes.TOCOPHOM1)
print("DM_avite1_c",solution.fluxes.DM_avite1_c)

EX_tyr__L_e -0.8597443803366394
PPND 0.0
TYRTA 0.7989720203366394
TOCOPHOM1 0.7989720203366394
DM_avite1_c 0.7989720203366394


It is shown here that in the absence of exogenous tyrosine, 3-(4-Hydroxyphenyl)pyruvate is produced in reaction PPND while reaction TYRTA has a negative flux meaning that a part of the 3-(4-Hydroxyphenyl)pyruvate pool is consumed to produce tyrosine. When tyrosine is supplied in the medium, the reaction PPND does not carry any flux anymore. In this case, 3-(4-Hydroxyphenyl)pyruvate is produced from tyrosine in the reaction TYRTA. For this reason, TYRTA could be a potential target for overexpresssion. To test this, a flux variability analysis is performed which shows the minimal and maximal flux that a reaction can carry.

In [13]:
from cobra.flux_analysis import flux_variability_analysis

with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    reactions_OE = [model.reactions.EX_tyr__L_e, model.reactions.TYRTA, model.reactions.PPND, model.reactions.DM_avite1_c]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

              minimum   maximum
EX_tyr__L_e -1.331703 -0.488684
TYRTA        0.427912  0.798973
PPND         0.000000  0.291163
DM_avite1_c  0.719075  0.798973


In an attempt to enforce a higher flux in the reaction TYRTA, the lower bound was raised to 0.8. However, this modification proofed to be infeasible. 

In [88]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    model.reactions.TYRTA.lower_bound = 0.8
    reactions_OE = [model.reactions.EX_tyr__L_e, model.reactions.TYRTA, model.reactions.PPND, model.reactions.DM_avite1_c]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

Infeasible: There is no optimal solution for the chosen objective! (infeasible).

In [89]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    model.reactions.TYRTA.lower_bound = 0.7
    reactions_OE = [model.reactions.EX_tyr__L_e, model.reactions.TYRTA, model.reactions.PPND, model.reactions.DM_avite1_c]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

              minimum   maximum
EX_tyr__L_e -1.331703 -0.760772
TYRTA        0.700000  0.798973
PPND         0.000000  0.077662
DM_avite1_c  0.719075  0.798973


Changing the lower bound of TYRTA to 0.7 to enforce a higher flux did not change the maximal flux of alpha-tocopherol production. 

### Flux Scanning Based On Enforced Objective Flux (FSEOF)

In order to find potential target genes for overexpression that could improve the production of alpha-tocopherol, Flux Scanning Based On Enforced Objective Flux (FSEOF) was performed. FSEOF is an algorithm that was created for this purpose. Constraint-based flux analysis is used to search for fluxes that rise when the formation of the desired product is enforced as the objective, while maximizing the flux into biomass formation at the same time [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2869140/].

In [14]:
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import cameo
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

In [16]:
model = read_sbml_model('model_atocopherol_tyr.xml')
model.objective = model.reactions.DM_avite1_c

In [3]:


import matplotlib.pyplot as plt
#import matplotlib.colors as mcolors

import numpy as np

In [5]:
fseof = FSEOF(model)

with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    result = fseof.run(target=model.reactions.DM_avite1_c)
    df = result.data_frame
df

,1,2,3,4,5,6,7,8,9,10
SHK3Dr,0.106920,0.106920,0.106920,0.106920,0.107109,0.107937,0.108764,0.109591,0.110418,0.111246
DHORTS,-0.145550,-0.145550,-0.145550,-0.145550,-0.145807,-0.146934,-0.148060,-0.149186,-0.150312,-0.151438
OMPDC,0.145550,0.145550,0.145550,0.145550,0.145807,0.146934,0.148060,0.149186,0.150312,0.151438
G5SD,0.097264,0.097264,0.097264,0.097264,0.097436,0.098189,0.098942,0.099694,0.100447,0.101199
APRAUR,0.009262,0.018328,0.027393,0.036459,0.043977,0.046275,0.048573,0.050872,0.053170,0.055468
...,...,...,...,...,...,...,...,...,...,...
TYRL,0.000098,0.000098,0.000098,0.000098,0.000098,0.000099,0.000100,0.000101,0.000101,0.000102
THZPSN3,0.000098,0.000098,0.000098,0.000098,0.000098,0.000099,0.000100,0.000101,0.000101,0.000102
4HTHRK,0.000294,0.000294,0.000294,0.000294,0.000295,0.000297,0.000299,0.000302,0.000304,0.000306
BIOMASS_Ec_iML1515_core_75p37M,0.440000,0.440000,0.440000,0.440000,0.440779,0.444183,0.447588,0.450992,0.454397,0.457801


The dataframe shows how the fluxes of the listed reations change with each step. For some reason the rate of the target reaction DM_avite1_c goes down with every step and not up as it would be expected.

The reactions that show no or a low change in flux in response to the enforced objective flux are not of interest when searching for overexpression target. Therefore, these reactions are filtered out.

In [7]:
# sorting the dataframe
df= df.sort_values([1], ascending = False)
 
from pytest import approx
# removing genes from dataframe 
removed_zero = (df != approx(0.0)).all(axis=1)
df_removed_zero = df.loc[removed_zero]
df_removed_zero

,1,2,3,4,5,6,7,8,9,10
ATPS4rpp,50.110096,50.245330,50.380565,50.515799,50.664717,50.859766,51.054814,51.249862,51.444910,51.639958
CYTBO3_4pp,32.104193,32.189447,32.274701,32.359955,32.452768,32.571060,32.689352,32.807644,32.925936,33.044228
NADH16pp,25.851745,25.942920,26.034094,26.125269,26.222714,26.341299,26.459884,26.578469,26.697054,26.815639
O2tpp,16.844895,16.879532,16.914169,16.948807,16.987226,17.038397,17.089567,17.140737,17.191908,17.243078
O2tex,16.844895,16.879532,16.914169,16.948807,16.987226,17.038397,17.089567,17.140737,17.191908,17.243078
...,...,...,...,...,...,...,...,...,...,...
GLUDy,-3.742610,-3.783405,-3.824201,-3.864997,-3.905376,-3.944355,-3.983333,-4.022311,-4.061289,-4.100267
Htex,-4.045583,-4.109043,-4.172503,-4.235963,-4.295632,-4.342521,-4.389410,-4.436299,-4.483189,-4.530078
PGK,-13.959111,-13.942821,-13.926530,-13.910239,-13.902330,-13.922677,-13.943024,-13.963370,-13.983717,-14.004064
H2Otpp,-37.191666,-37.359450,-37.527235,-37.695019,-37.851832,-37.971663,-38.091493,-38.211323,-38.331153,-38.450983


In the next step, the relative change in flux is calculated for the reactions and the most promising candidates for overexpression are selected by setting a cut off for the relative change in flux of 1.8.

In [12]:
df['relative_change_in_flux'] = 1 + (df[10]-df[1])/df[10]
target_reactions_df = df[df['relative_change_in_flux'] > 1.8]
target_reactions_df

,1,2,3,4,5,6,7,8,9,10,relative_change_in_flux
GART,0.028276,0.055474,0.082671,0.109868,0.132421,0.139320,0.146219,0.153118,0.160017,0.166916,1.830597
RBFSa,0.018524,0.036655,0.054787,0.072918,0.087953,0.092550,0.097147,0.101743,0.106340,0.110937,1.833022
DB4PS,0.018524,0.036655,0.054787,0.072918,0.087953,0.092550,0.097147,0.101743,0.106340,0.110937,1.833022
APRAUR,0.009262,0.018328,0.027393,0.036459,0.043977,0.046275,0.048573,0.050872,0.053170,0.055468,1.833021
DHPPDA2,0.009262,0.018328,0.027393,0.036459,0.043977,0.046275,0.048573,0.050872,0.053170,0.055468,1.833021
RBFSb,0.009262,0.018328,0.027393,0.036459,0.043977,0.046275,0.048573,0.050872,0.053170,0.055468,1.833021
PMDPHT,0.009262,0.018328,0.027393,0.036459,0.043977,0.046275,0.048573,0.050872,0.053170,0.055468,1.833021
GTPCII2,0.009262,0.018328,0.027393,0.036459,0.043977,0.046275,0.048573,0.050872,0.053170,0.055468,1.833021
FMNAT,0.009164,0.018230,0.027295,0.036361,0.043878,0.046176,0.048474,0.050771,0.053069,0.055366,1.834483
RBFK,0.009164,0.018230,0.027295,0.036361,0.043878,0.046176,0.048474,0.050771,0.053069,0.055366,1.834483


As a result, twelve reactions are listed that have the highest relative change in flux when running the FSEOF algorithm.

As the rate of the target reaction DM_avite1_c goes down with every iteration and not up as it would be expected, the applicability of the results of FSEOF to the objective to maximize the production of alpha-tocopherol is in question. For this reason, the potential target reactions were not investigated further.